# 🚀 Deployment Guide

Complete guide for deploying Hybrid-GCS in production.

**Topics:**
- Model serialization
- Hardware integration
- Real-time execution
- Safety considerations
- Monitoring & logging

## 1. Model Preparation

In [ ]:
import numpy as np
import json
from pathlib import Path
from hybrid_gcs.core import GCSDecomposer
from hybrid_gcs.integration import HybridPolicy, HybridPolicyConfig
from hybrid_gcs.utils import ConfigManager

# Setup deployment directory
deploy_dir = Path('deployment')
deploy_dir.mkdir(exist_ok=True)

print("✅ Deployment directory ready")

## 2. Model Serialization

In [ ]:
# Create deployment config
deployment_config = {
    'version': '2.0.0',
    'model_type': 'hybrid_gcs',
    'hybrid_config': {
        'blend_method': 'weighted',
        'gcs_weight': 0.5,
        'rl_weight': 0.5,
        'planning_horizon': 10,
        'safety_margin': 0.1,
    },
    'hardware': {
        'type': 'real_robot',
        'max_frequency': 100,  # Hz
        'control_mode': 'velocity',
    },
    'safety': {
        'enable_emergency_stop': True,
        'enable_collision_detection': True,
        'velocity_limit': 0.5,
        'acceleration_limit': 1.0,
    },
    'logging': {
        'level': 'INFO',
        'enable_file_logging': True,
        'log_frequency': 10,
    },
}

# Save config
with open(deploy_dir / 'config.json', 'w') as f:
    json.dump(deployment_config, f, indent=2)

print("✅ Deployment config saved")
print(json.dumps(deployment_config, indent=2))

## 3. Real-time Execution Template

In [ ]:
# Real-time control template
template = """
# Real-time Control Loop Template
# Place in deployment/control_loop.py

import numpy as np
import time
import logging
from hybrid_gcs.integration import HybridPolicy, HybridPolicyConfig
from hybrid_gcs.utils import ConfigManager

class RealTimeController:
    def __init__(self, config_path='config.json', log_path='logs'):
        self.config = ConfigManager(config_path)
        self.logger = logging.getLogger(__name__)
        
        # Initialize policy
        hybrid_config = HybridPolicyConfig(**self.config.get('hybrid_config'))
        self.policy = HybridPolicy(decomposer, rl_policy, hybrid_config)
        
        # Hardware interface
        self.robot = self._init_hardware()
        
    def _init_hardware(self):
        """Initialize robot hardware."""
        # TODO: Initialize actual hardware
        pass
    
    def run(self, target_frequency=100):
        """Main control loop."""
        period = 1.0 / target_frequency
        state_history = []
        
        while True:
            start_time = time.time()
            
            # 1. Get state from sensors
            state = self.robot.get_state()
            state_history.append(state)
            
            # 2. Compute action
            try:
                action = self.policy.compute_action(state, self.goal)
            except Exception as e:
                self.logger.error(f"Policy error: {e}")
                action = np.zeros(6)  # Safe fallback
            
            # 3. Apply safety limits
            action = self._apply_safety_limits(action)
            
            # 4. Send to robot
            self.robot.apply_action(action)
            
            # 5. Logging (sparse to avoid overhead)
            if len(state_history) % 10 == 0:
                self.logger.info(f"State: {state}, Action: {action}")
            
            # 6. Maintain frequency
            elapsed = time.time() - start_time
            if elapsed < period:
                time.sleep(period - elapsed)
    
    def _apply_safety_limits(self, action):
        """Apply safety constraints."""
        config = self.config.get('safety')
        
        # Velocity limits
        action = np.clip(action, -config['velocity_limit'], config['velocity_limit'])
        
        return action
    
    def stop(self):
        """Stop controller and cleanup."""
        self.robot.stop()
        self.logger.info("Controller stopped")
"""

# Save template
with open(deploy_dir / 'control_loop_template.py', 'w') as f:
    f.write(template)

print("✅ Real-time control loop template saved")

## 4. Safety Checklist

In [ ]:
safety_checklist = {
    'Pre-Deployment': [
        '✓ Model trained and validated',
        '✓ Safety limits configured',
        '✓ Emergency stop tested',
        '✓ Collision detection enabled',
        '✓ Hardware calibrated',
    ],
    'During Operation': [
        '✓ Real-time monitoring active',
        '✓ Logging enabled',
        '✓ Heartbeat signals working',
        '✓ Safety margins respected',
        '✓ Velocity limits enforced',
    ],
    'Post-Operation': [
        '✓ Logs analyzed',
        '✓ Performance metrics computed',
        '✓ Hardware checked',
        '✓ Model updated if needed',
        '✓ Incident report (if any)',
    ],
}

print("DEPLOYMENT SAFETY CHECKLIST")
print("=" * 50)

for phase, items in safety_checklist.items():
    print(f"\n{phase}:")
    for item in items:
        print(f"  {item}")

# Save as JSON
with open(deploy_dir / 'safety_checklist.json', 'w') as f:
    json.dump(safety_checklist, f, indent=2)

print(f"\n✅ Checklist saved to {deploy_dir / 'safety_checklist.json'}")

## 5. Monitoring & Logging Setup

In [ ]:
# Setup logging configuration
logging_config = {
    'version': 1,
    'disable_existing_loggers': False,
    'formatters': {
        'standard': {
            'format': '[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s'
        },
        'detailed': {
            'format': '[%(asctime)s] [%(name)s] [%(levelname)s] [%(filename)s:%(lineno)d] %(message)s'
        },
    },
    'handlers': {
        'console': {
            'class': 'logging.StreamHandler',
            'level': 'INFO',
            'formatter': 'standard',
            'stream': 'ext://sys.stdout'
        },
        'file': {
            'class': 'logging.handlers.RotatingFileHandler',
            'level': 'DEBUG',
            'formatter': 'detailed',
            'filename': 'logs/hybrid_gcs.log',
            'maxBytes': 10485760,  # 10MB
            'backupCount': 5
        },
    },
    'loggers': {
        'hybrid_gcs': {
            'level': 'DEBUG',
            'handlers': ['console', 'file'],
        }
    },
}

# Save logging config
import json
with open(deploy_dir / 'logging_config.json', 'w') as f:
    json.dump(logging_config, f, indent=2)

print("✅ Logging configuration saved")

## 6. Performance Monitoring

In [ ]:
# Performance metrics to monitor
monitoring_metrics = {
    'Real-time Performance': [
        'Control loop frequency (Hz)',
        'Latency (ms)',
        'Policy computation time (ms)',
        'Action execution time (ms)',
    ],
    'Safety Metrics': [
        'Min obstacle distance',
        'Emergency stops triggered',
        'Safety violations',
        'Collision events',
    ],
    'Performance Metrics': [
        'Task success rate',
        'Average reward',
        'Path efficiency',
        'Execution time vs expected',
    ],
    'System Health': [
        'CPU usage',
        'Memory usage',
        'Disk I/O',
        'Network latency (if remote)',
    ],
}

print("KEY METRICS TO MONITOR")
print("=" * 50)
for category, metrics in monitoring_metrics.items():
    print(f"\n{category}:")
    for metric in metrics:
        print(f"  • {metric}")

# Save metrics definition
with open(deploy_dir / 'monitoring_metrics.json', 'w') as f:
    json.dump(monitoring_metrics, f, indent=2)

print(f"\n✅ Monitoring metrics definition saved")

## 7. Deployment Summary

In [ ]:
deployment_summary = f"""
DEPLOYMENT SUMMARY
{'='*60}

✅ Created Files:
   • config.json - Deployment configuration
   • safety_checklist.json - Safety verification items
   • logging_config.json - Logging setup
   • monitoring_metrics.json - Performance metrics
   • control_loop_template.py - Real-time control template

📋 Key Steps:
   1. Review and customize config.json
   2. Implement control_loop.py based on template
   3. Setup hardware interface
   4. Run safety checklist
   5. Enable monitoring and logging
   6. Start deployment

🔒 Safety Features:
   • Emergency stop mechanism
   • Collision detection
   • Velocity/acceleration limits
   • Real-time logging
   • Health monitoring

📊 Monitoring:
   • Real-time performance metrics
   • Safety event logging
   • System health tracking
   • Performance analysis

🚀 Ready for Production!
"""

print(deployment_summary)

# Save summary
with open(deploy_dir / 'DEPLOYMENT_SUMMARY.txt', 'w') as f:
    f.write(deployment_summary)

print(f"\n✅ All deployment files ready in: {deploy_dir}")